# API Gateway Demo

In [ ]:
import os
from dotenv import load_dotenv

from data_prep import populate_bucket
from client_utils import (
    get_job_progress,
    get_job_results,
    get_overall_progress,
    update_job_results,
    submit_job,
)

load_dotenv()
api_key = os.environ["API_KEY"]
api_url = os.environ["API_URL"]
uploads_bucket = os.environ["UPLOADS_BUCKET"]

## Populate bucket

In [ ]:
normal_image_s3_uri, normal_metadata_s3_uri, normal_context_s3_uri = populate_bucket(
    bucket_name=uploads_bucket,
    image_fpath="../tests/data/normal_image.png",
    context="N/A",
    metadata="Title: Normal image",
)
offensive_image_s3_uri, offensive_metadata_s3_uri, offensive_context_s3_uri = populate_bucket(
    bucket_name=uploads_bucket,
    image_fpath="../tests/data/offensive_image.jpg",
    context="N/A",
    metadata="Title: Performers painted to look like negros",
)

## Create bias job

In [ ]:
bias_job_name = "demo_bias_job_7"
bias_job_objects = [
    {
        "work_id": "001_normal",
        "image_s3_uris": [normal_image_s3_uri, offensive_image_s3_uri],
        "context_s3_uri": offensive_context_s3_uri,
        "original_metadata_s3_uri": offensive_metadata_s3_uri,
    },
    {
        "work_id": "002_normal",
        "image_s3_uris": [normal_image_s3_uri],
    },
    {
        "work_id": "003_normal",
        "image_s3_uris": [offensive_image_s3_uri],
    },
]

submit_job(
    api_url=api_url,
    job_name=bias_job_name,
    job_type="bias",
    works=bias_job_objects,
    api_key=api_key,
)

## Check Bias Job Progress

In [ ]:
bias_progress = get_job_progress(
    api_url=api_url,
    job_name=bias_job_name,
    api_key=api_key,
)
for key, val in bias_progress.items():
    print(f"{key}: {val}")

## Get Bias Job Results

In [ ]:
item = get_job_results(
    api_url=api_url,
    job_name=bias_job_name,
    work_id=bias_progress["job_progress"]["READY FOR REVIEW"][0],
    api_key=api_key,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

item = get_job_results(
    api_url=api_url,
    job_name=bias_job_name,
    work_id=bias_progress["job_progress"]["READY FOR REVIEW"][1],
    api_key=api_key,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

## Create metadata job

In [ ]:
metadata_job_name = "demo_metadata_job_4"
metadata_job_objects = [
    {
        "work_id": "001_normal",
        "image_s3_uris": [normal_image_s3_uri, offensive_image_s3_uri],
        "context_s3_uri": offensive_image_s3_uri,
        "original_metadata_s3_uri": offensive_image_s3_uri,
    },
    {
        "work_id": "002_broken",
        "image_s3_uris": ["broken_uri"],
    },
]
submit_job(
    api_url=api_url,
    job_name=metadata_job_name,
    job_type="metadata",
    works=metadata_job_objects,
    api_key=api_key,
)

## Check metadata job progress

In [ ]:
metadata_progress = get_job_progress(
    api_url=api_url,
    job_name=metadata_job_name,
    api_key=api_key,
)
for key, val in metadata_progress.items():
    print(f"{key}: {val}")

## Get metadata job results

In [ ]:
item = get_job_results(
    api_url=api_url,
    job_name=metadata_job_name,
    work_id=metadata_progress["job_progress"]["READY FOR REVIEW"][0],
    api_key=api_key,
)
for key, val in item.items():
    print(f"{key}: {val}")
    print()

## Get overall progress

In [ ]:
overall_progress = get_overall_progress(api_url=api_url, api_key=api_key)
for key, val in overall_progress.items():
    print(f"{key}: {val}")

## Update Job Results

In [ ]:
update_job_results(
    api_url=api_url,
    job_name=bias_job_name,
    work_id=bias_progress["READY FOR REVIEW"][0],
    api_key=api_key,
)